In [1]:
import copy
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os
import time
import random
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 

In [2]:
driver = webdriver.Chrome(executable_path = '/Users/DM/FS/final/chromedriver')

In [3]:
def get_bio():
    bio = 'nothing'
    try:
        driver.find_element_by_class_name('oTruncateToggleText').click()
        d = driver.find_element_by_tag_name('o-profile-overview')
        bio = d.get_attribute('innerText')
    except:
        try:
            d = driver.find_element_by_tag_name('o-profile-overview')
            bio = d.get_attribute('innerText')
        except:
            bio = 'NA'
    return bio

In [4]:
def get_rate():
    rate = 'nothing'
    try:
        d = driver.find_element_by_tag_name('cfe-profile-rate').find_element_by_tag_name('span').find_element_by_tag_name('span')
        rate = d.text
    except:
        rate = 'NA'
    return rate

In [5]:
def get_headline():
    headline = 'nothing'
    try:
        d = driver.find_element_by_class_name('fe-job-title').find_element_by_tag_name('span')
        headline = d.text
    except:
        headline = 'NA'
    return headline

In [6]:
def get_skills():
    skills = 'nothing'
    try:
        d = driver.find_element_by_class_name('o-profile-skills').find_element_by_tag_name('span').find_elements_by_class_name('o-tag-skill')
        skills = [item.text for item in d]
    except:
        skills = 'NA'
    return skills

In [7]:
def get_tests():
    tests = 'nothing'
    try:
        d = driver.find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')
        tests = len(d)
    except:
        tests = 'NA'
    return tests

In [8]:
def get_edu():
    edu = 'nothing'
    try:
        d = driver.find_element_by_tag_name('o-profile-education').find_element_by_tag_name('ul').find_elements_by_tag_name('li')
        edu = len(d)
    except:
        edu = 'NA'
    return edu

In [9]:
def get_exp():
    exp = 'nothing'
    try:
        driver.find_element_by_tag_name('o-items-limiter').find_element_by_tag_name('a').click()
    except:
        pass
    try:
        d = driver.find_element_by_tag_name('o-profile-employment-history').find_element_by_tag_name('ul').find_elements_by_tag_name('li')
        exp = len(d)
    except:
        exp = 'NA'
    return exp

In [10]:
def get_last_three():
    three = {'hours': 'NA',
            'jobs': 'NA',
            'earned': 'NA'}
    switcher = {'Total earned': 'earned',
               'Jobs': 'jobs',
               'Job': 'jobs',
               'Hours worked': 'hours'}
    d = driver.find_element_by_class_name('cfe-aggregates').find_element_by_tag_name('ul').find_elements_by_tag_name('li')
    numb = len(d)
    for i in range(1, numb):
        e = d[i]
        label = e.find_element_by_class_name('text-muted').text
        key_name = switcher[label]
        three[key_name] = e.find_element_by_tag_name('h3').text
    return three

In [11]:
def get_person_details():
    person = get_last_three()
    time.sleep(1)
    person['bio'] = get_bio()
    time.sleep(1)
    person['rate'] = get_rate()
    person['headline'] = get_headline()
    person['skills'] = get_skills()
    time.sleep(1)
    person['exp'] = get_exp()
    time.sleep(1)
    person['edu'] = get_edu()
    person['tests'] = get_tests()
    return person

In [12]:
def page_of_ten_scrape():
    people = []
    for i in range(1, 11):
        try:
            driver.find_element_by_xpath('//*[@id="layout"]/div[2]/div[2]/div/div/div/div/div/freelancers-search/div/div[2]/section[' + str(i) + ']/div/div/article/div').click()
            time.sleep(2)
            p = get_person_details()
#             print("trying to get person " + str(i))
            time.sleep(2)
            people.append(p)
            driver.find_element_by_xpath('/html/body/div[3]/div/div/div[1]/button').click()
            time.sleep(2)
        except:
            pass
    return people

# https://www.upwork.com/ab/profiles/search/details/~01b75b68f5da091112/profile?sqp=revenue%3D0%26pt%3Dindependent%26last%3D30%26user_pref%3D2%26page%3D160

In [13]:
# def get_all(pages_to_scrape = 50):
#     peeps = []
#     randos = random.sample(range(5, 499), pages_to_scrape)
#     for i in randos:
#         print("trying new page")
#         try:
#             driver.get('https://www.upwork.com/ab/profiles/search/?pt=independent&user_pref=2&page=' + str(i))
#             time.sleep(2.1)
#             pp = page_of_ten_scrape()
#             peeps += pp
#             time.sleep(1.8)
#         except:
#             break
#     return peeps

In [14]:
def get_all_click(pages_to_scrape = 50):
    peeps = []
    for i in range(pages_to_scrape):
        print("page: " + str(i))
        try:
            driver.find_element_by_class_name('pagination-next').find_element_by_tag_name('a').click()
            time.sleep(2)
            pp = page_of_ten_scrape()
            peeps += pp
            time.sleep(2)
        except:
            break
    return peeps

In [17]:
try3 = get_all_click(pages_to_scrape = 10)

page: 0
page: 1
page: 2


In [ ]:
len(try3)

In [ ]:
df = pd.DataFrame(try3)

In [ ]:
df.head(7)

In [ ]:
df.to_csv("alllast.csv")

In [ ]:
# columns = set()
# for i in range(len(df)):
#     columns.update(df['skills'].iloc[i])

In [ ]:
# def get_ten_rates():
#     rates = []
#     for i in range(2,12):
#         time.sleep(2)
#         rate = driver.find_element_by_xpath('//*[@id="layout"]/div[2]/div[2]/div/div/div/div/div/freelancers-search/div/div[2]/section[' + str(i) +']/div/div/article/div/div[1]/div/div[4]/div[1]/div[1]/div/strong')
#         rates.append(rate.text)
#     return rates

In [ ]:
# def get_ten_headlines():
#     headlines = []
#     for i in range(2,12):
#         time.sleep(2)
#         headline = driver.find_element_by_xpath('//*[@id="layout"]/div[2]/div[2]/div/div/div/div/div/freelancers-search/div/div[2]/section[' + str(i) +']/div/div/article/div/div[1]/div/div[2]/div[2]/div/div/div[2]/strong')
#         headlines.append(headline.text)
#     return headlines

In [ ]:
# [item.text for item in d.find_elements_by_tag_name('span')]